## **Calculations for integrated LCA**

In [ ]:
import os
import sys
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

from IntLCA import intLCA
from IntLCA.utils.fetcher import ActivityFilter

##### **1. Access files**

In [ ]:
project = "Integrated LCA project"
scenarios = ["SSP2-Base", "SSP2-PkBudg1150", "SSP2-PkBudg500"]
years = [2020, 2025, 2030, 2035, 2040, 2045, 2050]
base_path = "C:/Users/mcharalam/dev/prospectiveLCA/Premise/pLCA/Enhanced-Technosphere-wide-LCA-paper/Integrated_LCA/Notebooks/Setting up/export" #--> Change this to the path where the data is stored
directory = "Results"
methods = ["IPCC 2021"]

In [ ]:
# Instantiate the IntLCA class
int_lca = intLCA.IntLCA(project, scenarios, years, base_path, directory, methods)

In [ ]:
# Access the csv files and return a list of paths. 
paths = int_lca.access_data()

##### **2. User specifications**

- *Activities to run calculations:*

In [ ]:
keys_to_run = [
    {"starts": "transport, freight, lorry", "includes_all": ["diesel", "40t", "long haul"], "location": "World"}
]

- *Activities to add:*

In [ ]:
#keys for the activity filter - The following activities will be added one at a time.
#Each dictionary represents a set of activities with the same name but for all the IAM activities. 
#We will update the current market for diesel ("World") which is in our foreground activity to inlcude the following activities, while keeping the regional fractions (This is an assumption)
keys_to_add = [
    {"starts": "diesel production,--", "includes_all": ["PEM", "DAC", ">3MW"], "excluding_locations": ["GLO", "RER"]},
    {"starts": "diesel production,--", "includes_all": ["wet poplar", "DAC", "CCS"], "excluding_locations": ["GLO", "RER"]}
]

- *Activities to modify:*

In [ ]:
keys_to_modify = [
        {"includes_all": ["diesel production, synthetic, from Fischer Tropsch process", "energy allocation"]},
        {"includes": "market for diesel, low-sulfur", "excludes": "period", "location": "World"}
]


- *Activities to replace:*

In [ ]:
keys_to_replace = [
    {"includes_all": ["Diesel, synthetic", "energy allocation"]},
    {"includes": "market for diesel, low-sulfur", "excludes":"period", "excluding_locations": "World"}
]

##### **3. Run calculations**

In [ ]:
FU = 1

for path in paths: #--> For each chosen path
    A, B, A_inds, B_inds, A_inds_rev, B_inds_rev = int_lca.create_matrices(path)
    exc_to_remove, act_to_modify, act_to_add, act_to_run = int_lca.filtering(A_inds, keys_to_add, keys_to_modify, keys_to_replace, keys_to_run)
    # Create a dictionary with the unique names of H2 and CO2
    unique_names_h2 = set(A_inds_rev[i][0].split("--")[2] for i in act_to_add)
    unique_names_co2 = set(A_inds_rev[i][0].split("--")[1] for i in act_to_add)
    name_dict_h2 = {}
    for name in unique_names_h2:
        name_dict_h2[name] = 0
    name_dict_co2 = {}
    for name in unique_names_co2:
        name_dict_co2[name] = 0

    # Remove the current activities from the technosphere that we will replace
    A_ = int_lca.copy_technosphere(A)
    A_ = int_lca.modify_technosphere_remove(exc_to_remove, act_to_modify)

    storing_all_diesel = []
    for nameh2 in name_dict_h2:
        for nameco2 in name_dict_co2:
            for i in act_to_add:
                if A_inds_rev[i][0].split("--")[1] == nameco2 and A_inds_rev[i][0].split("--")[2] == nameh2: #Since we will have multiple names with several regions we make sure we only take the activities with the same name
                    act_tech = A_inds_rev[i] 
                    for d in act_to_modify: #<---- Changing the background
                        if "energy allocation" in A_inds_rev[i][0] and A_inds_rev[i][3] == A_inds_rev[d][3]:  # The activity we add needs to be in the same location with the activity we modify
                            exc_remove = [exc_remove for e in exc_to_remove if "energy allocation" in A_inds_rev[e][0]
                                          and A_inds_rev[e][3] == A_inds_rev[d][3]]
                            A_ = int_lca.modify_technosphere_add(exc_remove[0], d, i)

                        if "market for diesel, low-sulfur" in A_inds_rev[i][0] and A_inds_rev[i][3] == A_inds_rev[d][3]:
                            exc_remove = [exc_remove for e in exc_to_remove if "market for diesel, low-sulfur" in A_inds_rev[e][0]]
                            for dm in exc_remove:
                                A_ = int_lca.modify_technosphere_add(dm, d, i)     #MOdify the foreground to include onlu the production pathway of choice
                                
#----------- Calculation ---------------
            results = []
            for idx in act_to_run:
                if A_inds_rev[idx][-1] == A_inds_rev[idx][0].split(",")[5].split(" ")[1]:  # Choosing only the truck activity constructed in the year of the IAM
                    act_truck = A_inds_rev[idx]
                    dfs = int_lca.breakdown(idx, FU, results, act_tech)
#---------------------------------------

            for i in act_to_add:
                if A_inds_rev[i][0].split("--")[1] == nameco2 and A_inds_rev[i][0].split("--")[2] == nameh2: #Since we will have multiple names with several regions we make sure we only take the activities with the same name
                    act_tech = A_inds_rev[i]  
                    for d in act_to_modify: 
                        if "energy allocation" in A_inds_rev[i][0] and A_inds_rev[i][3] == A_inds_rev[d][3]: 
                            A_ = int_lca.modify_technosphere_remove(i, d)
                                    
                        if "market for diesel, low-sulfur" in A_inds_rev[i][0] and A_inds_rev[i][3] == A_inds_rev[d][3]:
                            A_ = int_lca.modify_technosphere_remove(i, d) 

df_STORE = pd.concat(dfs)
df_STORE = df_STORE.reset_index()
df_STORE = df_STORE.drop(columns=["index"])
df_STORE = int_lca.organize_results(df_STORE)

for col in df_STORE.columns:
    if "transport, freight, lorry," in col:
        df_STORE.rename(columns={str(col): "transport activity"}, inplace=True)
    if "Heavy duty truck" in col:
        df_STORE.rename(columns={str(col): "Heavy duty truck"}, inplace=True)
    if 'market for diesel' in col:
        df_STORE.rename(columns={str(col): "Fuel market"}, inplace=True)

for col in df_STORE.columns:
    if 'transport activity' in col:
        df_new = df_STORE.loc[:, col]
    if 'Heavy duty truck' in col:
        df_tr = df_STORE.loc[:, col]
#
df_tot = df_new.iloc[:, :].sum(axis=1)
col_name = str(set(df_new.columns)).split('{')[1].split('}')[0]
df_tot = df_tot.to_frame()
df_tot[col_name] = df_tot.iloc[:, :]
df_tot = df_tot.reset_index(drop=True)
df_tot = df_tot.iloc[:, 1]

df_truck = df_tr.iloc[:, :].sum(axis=1)
col_name = str(set(df_tr.columns)).split('{')[1].split('}')[0]
df_truck = df_truck.to_frame()
df_truck[col_name] = df_truck.iloc[:, :]
df_truck = df_truck.reset_index(drop=True)
df_truck = df_truck.iloc[:, 1]

columns_to_drop = []
for col in df_STORE.columns:
    if "transport activity" in col:
        columns_to_drop.append(col)
    if "Heavy duty truck" in col:
        columns_to_drop.append(col)
columns_to_drop = set(columns_to_drop)

for column in columns_to_drop:
    if column in df_STORE.columns:
        df_STORE.drop(column, axis=1, inplace=True)

dfs_toadd = [df_tot, df_truck, df_STORE]
df_final = pd.concat(dfs_toadd, axis=1)


##### **4. Save results**

In [ ]:
int_lca.save_results("Results", df_final)